# Experiments with Synthetic Data using a formulation of the Swing-Equation

In [ ]:
# Add the 'src' as root folder, to find other modules in the project
import sys
sys.path.append("../")

In [ ]:
# local imports
from simulator.swing_equation import swing_equation

# external imports
import torch
import os
import pandas as pd
import numpy as np

from sbi.inference import SNPE, simulate_for_sbi, prepare_for_sbi
from sbi.utils import BoxUniform, posterior_nn
from matplotlib import pyplot as plt

# Define Training Parameters

1. [Simulation Parameters](#simulation-parameters)
2. [Neural Density Estimator Parameters](#neural-density-estimator-parameters)


## Simulation Parameters

In [ ]:
# Define the simulation parameters
# Define length and timesteps of the simulations
dt = 0.01
T = 10
simulator = swing_equation

# Define a proposal for parameter distributions
proposal_low = torch.tensor([0.1, 1.0, 0.1, 40.0, 0.1])
proposal_high = torch.tensor([10.0, 100.0, 10.0, 70.0, 10.0])

proposal = BoxUniform(low=proposal_low, high=proposal_high)


num_simulations = 20000
num_workers = 20
simulation_batch_size = 1
seed = None
show_progress_bar = True

## Neural Density Estimator Parameters

In [ ]:


# Define  


# Define output directories for results
dir_path_plots = '../../results/simulation_length/plots/'
dir_path_tables = '../../results/simulation_length/csv/'

# Create the directories
os.makedirs(dir_path_plots, exist_ok=True)
os.makedirs(dir_path_tables, exist_ok=True)



prior = 

# Simulation parameters


# Density estimator parameters
model = 'maf'
hidden_features = 50
num_transforms = 10
z_score_x = 'independent'
z_score_theta = 'independent'
num_bins = 10
num_components = 10

# Define the true parameters for training and evaluation
theta_true = torch.tensor([4.0, 40.0, 3.0, 60.0, 7.0])

# Define the simulators with different timesteps and observations from them
sim_ops = []

for dt in dts:
    
    # Wrap the simulator to allow timestep change
    def wrap_simulator(theta, dt=dt):
        return swing_equation(theta, dt=dt, T=T)
    
    # Generate observation for the current simulator function
    observation = wrap_simulator(theta_true)
    
    # Append simulator function, observation, and total length of the simulation
    sim_ops.append((wrap_simulator, observation, T / dt))